In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import math
import re
import pandas as pd

In [6]:
#성별과 연령별로 이미지 파일을 사전에 저장

dic = {}

for i in glob.glob("./data/BoneAge/*/*.jpg"):
    if i.split("\\")[-2] not in dic.keys():
    
        dic[i.split("\\")[-2]] = [i.split("\\")[-1]]
    else:
        dic[i.split("\\")[-2]].append(i.split("\\")[-1])

for i in glob.glob("./data/BoneAge/*/*/*.jpg"):
    if i.split("\\")[-2] not in dic.keys():
        
        dic[i.split("\\")[-2]] = [i.split("\\")[-1]]
    else:
        dic[i.split("\\")[-2]].append(i.split("\\")[-1])




In [29]:
#이미지들이 사전에 잘 들어 갔는지 확인
dic.keys()

dict_keys(['BA_F_10yr', 'BA_F_11yr', 'BA_F_12yr', 'BA_F_13yr', 'BA_F_14yr', 'BA_F_15yr', 'BA_F_16yr', 'BA_F_3yr', 'BA_F_4yr', 'BA_F_5yr', 'BA_F_6yr', 'BA_F_7yr', 'BA_F_8yr', 'BA_F_9yr', 'BA_M_10yr', 'BA_M_11yr', 'BA_M_12yr', 'BA_M_13yr', 'BA_M_14yr', 'BA_M_15yr', 'BA_M_16yr', 'BA_M_17yr', 'BA_M_3yr', 'BA_M_4yr', 'BA_M_5yr', 'BA_M_6yr', 'BA_M_7yr', 'BA_M_8yr', 'BA_M_9yr'])

In [30]:
#직접 roi를 하기 위한 마우스 클릭 이벤트 함수
def mouse_event(event, x, y, flags, param):
    
    global tmp_
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(param, (x, y), 2, (255, 255, 255), 10)
        tmp_.append([x,y])
        print(x,y)
        cv2.imshow("image",param) 
       

In [31]:
#마우스 클릭을 위해 img 를 띄어주는 함수 (사실 큰 필요 x) 
def get_iou_points(img):
    
    cv2.namedWindow('image', flags=cv2.WINDOW_NORMAL)
    cv2.resizeWindow(winname='image', width=800, height=800)
    cv2.imshow("image", img)
    cv2.setMouseCallback("image", mouse_event, img)
    cv2.waitKey(0)

In [148]:
#이미지를 roi에 따라 자르는 함수
def crop_img(img,roi_1,roi_2,roi_3,roi_4,roi_5):

    cropped_roi_1_img = img[roi_1[0][1]:roi_1[1][1],roi_1[0][0]:roi_1[1][0]]
    cropped_roi_2_img = img[roi_2[0][1]:roi_2[1][1],roi_2[0][0]:roi_2[1][0]]
    cropped_roi_3_img = img[roi_3[0][1]:roi_3[1][1],roi_3[0][0]:roi_3[1][0]]
    """
    cropped_roi_4_img = img[roi_4[0][1]:roi_4[1][1],roi_4[0][0]:roi_4[1][0]]
    cropped_roi_5_img = img[roi_5[0][1]:roi_5[1][1],roi_5[0][0]:roi_5[1][0]]
    """
    
    
    optimzed_imgs = optimze_img(cropped_roi_1_img,cropped_roi_2_img,cropped_roi_3_img,None,None)
    return optimzed_imgs

In [172]:
#잘라낸 이미지를 평탄화 + 정규화 -> 노이즈 제거후 이진화
def optimze_img(r1,r2,r3,r4,r5):
  
    tmp_binary_img = []
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)) #CLAHE 생성
    try : 
        for _,img in enumerate([r1,r2,r3,r4,r5]):

            if img is not None:
                
                resized_img = cv2.resize(img,(256,256))
                blured_img = cv2.GaussianBlur(resized_img,(5,5),0)

                if len(blured_img.shape) == 3:
                    blured_img = blured_img[:,:,0]
                else:
                    blured_img = blured_img

                


                clahed_img = clahe.apply(blured_img)          #CLAHE 적용


                _,binary_img = cv2.threshold(clahed_img,clahed_img.mean()*1.25,255,cv2.THRESH_BINARY)
                
                tmp_binary_img.append(binary_img)
                
                
        
        
    except:
        tmp_binary_img = []
    return np.array(tmp_binary_img)
    

In [34]:
#출력함수
def show(img):
    plt.figure(figsize=(15, 10))
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap = "gray")
    plt.show()

In [35]:
#각도를 받아서 이미지를 회전하는 함수
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR,borderValue=(255,255,255))
    return result

In [36]:
#자동 roi 와 내가 선택한 roi의 좌표들을 가지고 iou를 구하는 함수
def get_iou(roi_points,iou_points):
    iou_cor,iou_four,iou_mid = 0,0,0
    
    intersection_cor_x = min(roi_points[0][1][0],iou_points[0][1][0])- max(roi_points[0][0][0],iou_points[0][0][0])
    intersection_cor_y = min(roi_points[0][1][1],iou_points[0][1][1])- max(roi_points[0][0][1],iou_points[0][0][1])
    
    cor_area = (roi_points[0][1][0] - roi_points[0][0][0]) * (roi_points[0][1][1] - roi_points[0][0][1]) + (iou_points[0][1][0] - iou_points[0][0][0]) * (iou_points[0][1][1] - iou_points[0][0][1])
    intersection_cor_area = intersection_cor_x * intersection_cor_y
    
    intersection_four_x = min(roi_points[1][1][0],iou_points[1][1][0])- max(roi_points[1][0][0],iou_points[1][0][0])
    intersection_four_y = min(roi_points[1][1][1],iou_points[1][1][1])- max(roi_points[1][0][1],iou_points[1][0][1])
    
    four_area = (roi_points[1][1][0] - roi_points[1][0][0]) * (roi_points[1][1][1] - roi_points[1][0][1]) + (iou_points[1][1][0] - iou_points[1][0][0]) * (iou_points[1][1][1] - iou_points[1][0][1])
    intersection_four_area = intersection_four_x*intersection_four_y
    
    intersection_mid_x = min(roi_points[2][1][0],iou_points[2][1][0])- max(roi_points[2][0][0],iou_points[2][0][0])
    intersection_mid_y = min(roi_points[2][1][1],iou_points[2][1][1])- max(roi_points[2][0][1],iou_points[2][0][1])
    
    mid_area = (roi_points[2][1][0] - roi_points[2][0][0]) * (roi_points[2][1][1] - roi_points[2][0][1]) + (iou_points[2][1][0] - iou_points[2][0][0]) * (iou_points[2][1][1] - iou_points[2][0][1])
    intersection_mid_area = intersection_mid_x*intersection_mid_y
    
    if intersection_cor_area > 0 and intersection_four_area >0 and intersection_mid_area>0:
        iou_cor = intersection_cor_area / (cor_area - intersection_cor_area)
        iou_four = intersection_four_area / (four_area-intersection_four_area)
        iou_mid = intersection_mid_area / (mid_area - intersection_mid_area)
    
    return iou_cor,iou_four,iou_mid
    
    

In [173]:
no_pattern = re.compile("([0-9]+_M|[0-9]+)")

#이미지 넘버 별로 X값 저장을 위한 데이터 프레임 생성

X_dic = {}
#iou 좌표들을 수집한뒤 나이별로 사전으로 정리
dic_iou_points = {}

#사전에 있는 값들로 이미지 출력
for year in dic.keys():
    img_path = "./data/BoneAge/"
    if "M" in year:
        pa = year +"/"+year
    else:
        pa = year
    
    for n,i in enumerate(dic[year]):
        img_key = re.findall(no_pattern,i)[0]
        
        if n == 0:
            img_path += pa+"/"
            print(img_path)
        else:
            img = cv2.imread(img_path+i)

            r_img_ = np.copy(img)
            img_ = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            height, width = img_.shape
            img_ = img_[0:(int)(height*0.9),0:(int)(width*0.95)]
            ret, img = cv2.threshold(img_, img.mean()*0.99, 255, cv2.THRESH_BINARY)

            #좌표 저장공간 설정(첫번째 공간에는 무게중심점, 두번째 공간에는 start점, 세번째에는 far점) 
            dots = [[],[],[]]

            contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            max_cnt = max(contours, key=cv2.contourArea)

            mask = np.zeros(img.shape, dtype=np.uint8)

            #컨투어 구하고 구한 컨투어로 이미지 그리기
            cv2.drawContours(mask, [max_cnt], -1, (255,255,255), -1)

            #이미지 팽창
            k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
            mask = cv2.dilate(mask,k)
            

            #회전하기 위해서 가운데 손가락이 가장 길기때문에 가장 먼저 흰색을 나타내는 좌표를 찾아 낸다.
            first_255_x_point = 0
            first_255_y_point = 0

            contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            max_cnt = max(contours, key=cv2.contourArea)


            #무게중심점 구하기 일단 
            M = cv2.moments(max_cnt)
            center = ( int(M['m10'] / M['m00']), int(M['m01'] / M['m00']) )

            for y,x_r in enumerate(mask) :
                if 255 in x_r:
                    x_255_indexs = np.where(x_r == 255)[0]

                    x_255_mid_index = x_255_indexs[(int)(len(x_255_indexs)/2)]
                    first_255_x_point = x_255_mid_index

                    first_255_y_point = y
                    break
           

            #width 의 중간값의 x값과 가장 먼저 흰색을 나타내는 좌표의 x값을 비교한다.
            #그리고 center 값을 기준으로 회전한다.

            half_h, half_w = center[1], center[0]
            ry = center[1]-first_255_y_point 
            rx = abs(first_255_x_point-half_w)
            radian = math.atan2(ry,rx)
            degree = 90 - (radian * 180 / math.pi)
            print("회전각도 : ", degree)

            if degree > 3:
                if first_255_x_point < half_w :
                    mask = rotate_image(mask,360-degree) 
                    r_img_ = rotate_image(img_,360-degree)
                else:
                    mask = rotate_image(mask,degree) 
                    r_img_ = rotate_image(img_,degree) 


            #이미지 외부 컨투어 구하기
            contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            max_cnt = max(contours, key=cv2.contourArea)


            #무게중심점 구하기 일단 
            M = cv2.moments(max_cnt)
            center = ( int(M['m10'] / M['m00']), int(M['m01'] / M['m00']) )


            #cv2.circle(img_,center,0,[255,255,255],10)
            dots[0].append(center)
            # 볼록한 지점 구하기
            hull = cv2.convexHull(max_cnt, returnPoints = False)
            hull1 = cv2.convexHull(max_cnt)
            # 오목한 지점 구하기
            defects = cv2.convexityDefects(max_cnt, hull)

            #거리를 저장 할 수 있는 공간 생성
            di =[]

            for index in range(defects.shape[0]):
                #시작점,끝점,far점,거리 할당
                start_point,end_point,far_point,distance = defects[index,0]

                far = tuple(max_cnt[far_point][0])
                start = tuple(max_cnt[start_point][0])
                end = tuple(max_cnt[end_point][0])
                
                #cv2.circle(r_img_,far,2,[255,255,255],10)
                #cv2.circle(r_img_,start,2,[255,255,255],10)

                #cv2.circle(r_img_,end,2,[255,255,255],10)
                #거리를 저장
                di.append(distance)
                dots[1].append(start)
                dots[2].append(far)

            #더 쉽게 불러오기 위해서 far, start로 분리
            far_pt = np.array(dots[2])
            start_pt = np.array(dots[1])


            far_xrange =[]
            far_yrange = []
            far_miny = 1000
            far_maxy = 0
            start_miny = 1000
            #가장 오목하게 들어가 있는 부분을 찾기 위해서 sort(내림차순)
            di =np.array(di)
            s_di=np.sort(di)[::-1]
            #내림차순된 거리들을 6개만 뽑아내기 위해서 slice
            for i in list(s_di[:6]):
                index = np.where(di ==i)[0]
                #6개의 좌표들 중에서 가장 최저의 y 값을 찾는다. (손목쪽 roi에 필요)
                far_miny = min(far_miny,far_pt[index[0]][1])
                #6개의 좌표들 중에서 가장 최고의 y 값을 찾는다.
                far_maxy = max(far_maxy,far_pt[index[0]][1])

                #가장 오목한 지점 6개의 좌표를 출력
                #cv2.circle(img_,np.array(far_pt[index[0]]),2,[255,255,255],5)

                #좌표들이 x,y로 나눠져 있어서 쉽게 비교하기 위해서 x,y끼리 나눈다.
                far_xrange.append(far_pt[index[0]][0])
                far_yrange.append(far_pt[index[0]][1])

            #far_xrange를 오름차순으로 정렬
            sorted_far_xrange = np.sort(far_xrange)
            sorted_far_yrange = np.sort(far_yrange)

            carpus_start_point = ((int)(np.sort(far_xrange)[0]),(int)(center[1]*1.05))
            carpus_end_point = (np.sort(far_xrange)[-1],(int)(far_maxy*1.09))
            #손목뼈 부분 roi 를 하기 위해서는 가장 오목하게 들어가 있는 부분중에서 가장 최저 x값(xrange[0])과 center값의 y값을 시작점으로
            #끝점으로는 가장 오목하게 들어가 있는 부분들 중에서 가장 최고 x값(xrange[-1])과 오목하게 들어간 점중에서 가장 최고 y값을 준다.
            #cv2.rectangle(r_img_,carpus_start_point,carpus_end_point,[255,255,255],5)

            print("==================================================================")
            print("손목 관절 roi 추출 시작좌표 : ",carpus_start_point)
            print("손목 관절 roi 추출 끝 좌표 : ",carpus_end_point)

            #손목뼈 위쪽에 있는 관절 4개를 추출하기 위해서는 오목하게 들어가 있는 부분중에서 가장 최저 x값(xrange[0])과 far_miny 값을 y값으로
            #끝점으로는 가장 오목하게 들어가 있는 부분들 중에서 가장 최고 x값(xrange[-1])과 center의 y값을 준다.

            four_start_point = ((int)(np.sort(far_xrange)[0]*0.85),far_miny)
            four_end_point = (np.sort(far_xrange)[-1],(int)(center[1]*1.05))
            #cv2.rectangle(r_img_,four_start_point,four_end_point,[255,255,255],5)

            print("==================================================================")
            print("가운데 4개 관절 roi 추출 시작좌표 : ",four_start_point)
            print("가운데 4개 관절 roi 추출 끝 좌표 : ",four_end_point)
            #가운데 손가락을 추출하기 위해서 start_pt의 x 좌표가 sorted_far_xrange 에서 3번째와 4번째 값 사이에 있어야 한다.
            middle_index = np.where((start_pt[:,0]<=(int)(sorted_far_xrange[3])) & (start_pt[:,0]>=sorted_far_xrange[2]))[0]
            #위에 조건식으로 나온 인덱스를 start_point에 대입하면 만족하는 좌표들이 여러개 나올것이다.
            middle_points = start_pt[middle_index]


            for point in middle_points:
                #가운데 손가락 사이에 있는 좌표들 중에서 최저 y 값을 찾는다.
                start_miny = min(start_miny,point[1])


            
            start_maxy = max(far_yrange[np.where(far_xrange == sorted_far_xrange[3])[0][0]],far_yrange[np.where(far_xrange == sorted_far_xrange[2])[0][0]])
            #시작 좌표로는 x값으로 sorted_far_xrange 에서 3번째와 y 값으로는 최저y 값을 주고 
            #마지막 좌표로는 x값으로 sorted_far_xrange 에서 4번째와 y값으로는 3번째 와 4번째 좌표의 최고 y 값을 준다.
            middle_finger_start_point = (sorted_far_xrange[2],start_miny)
            middle_finger_end_point = (sorted_far_xrange[3],start_maxy)
            
            
            print("==================================================================")
            print("가운데 손가락 roi 추출 시작좌표 : ",middle_finger_start_point)
            print("가운데 손가락 roi 추출 끝 좌표 : ",middle_finger_end_point)
            #cv2.rectangle(r_img_,middle_finger_start_point,middle_finger_end_point,[255,255,255],5)
            
            """
               #새끼손가락 시작점 x좌표
            little_finger_x = ()
            little_finger_y = ()

            min_little_finger_x = hull1[0][0][0]
            for i in range(1, len(hull1)):
                if min_little_finger_x > hull1[i][0][0]:
                    min_little_finger_x = hull1[i][0][0]
                    little_finger_x, little_finger_y = hull1[i][0]

            #엄지손가락 시작점 좌표
            thumb_x = ()
            thumb_y = ()

            max_thumb_x = hull1[0][0][0]
            for i in range(1,len(hull1)):
                if max_thumb_x < hull1[i][0][0]:
                    max_thumb_x = hull1[i][0][0]            
                    thumb_x, thumb_y = hull1[i][0] 
                    
                    
            # 새끼손가락 endpoint x 좌표
            little_finger_endpoint_x =[]
            for x, y in zip(far_xrange, far_yrange):
                if y < int(center[1]*0.9) :
                    little_finger_endpoint_x.append(x)

            # 엄지손가락 endpoint
            thumb_endpoint_list =[]
            for x, y in zip(far_xrange, far_yrange):
                if y > int(center[1]):
                    thumb_endpoint_list.append([x,y])

            thumb_endpoint =[]
            max_x = thumb_endpoint_list[0][0]
            if len(thumb_endpoint_list) == 1:
                thumb_endpoint = thumb_endpoint_list
            for i in range(1, len(thumb_endpoint_list)):       
                if max_x < thumb_endpoint_list[i][0]:
                    max_x = thumb_endpoint_list[i][0]
                    thumb_endpoint.append(thumb_endpoint_list[i])
                else:
                    thumb_endpoint.append(thumb_endpoint_list[0])
            
             #새끼손가락 ROI
            little_finger_start_point = (int(little_finger_x*0.5), int(little_finger_y * 0.8))
            little_finger_end_point = (min(little_finger_endpoint_x),(int)(center[1]*1.05))
            cv2.rectangle(r_img_,little_finger_start_point, little_finger_end_point,[255,255,255],2)
            print("==================================================================")
            print("새끼손가락 roi 추출 시작좌표 : ",little_finger_start_point)
            print("새끼손가락 roi 추출 끝 좌표 : ",little_finger_end_point)


            #엄지손가락 ROI
            thumb_start_point = (int(thumb_endpoint[0][0]*1.1), int(thumb_y*0.9))
            thumb_end_point = (int(thumb_x * 1.02), int(thumb_endpoint[0][1]))
            cv2.rectangle(r_img_,thumb_start_point, thumb_end_point,[255,255,255],2)
            print("==================================================================")
            print("엄지손가락 roi 추출 시작좌표 : ",thumb_start_point)
            print("엄지손가락 roi 추출 끝 좌표 : ",thumb_end_point)
            """
        
            
            optimzed_imgs = crop_img(r_img_,(carpus_start_point,carpus_end_point),(four_start_point, four_end_point),(middle_finger_start_point,middle_finger_end_point),None,None)
            
            if len(optimzed_imgs) > 0:
                
                X_dic[img_key] = optimzed_imgs
                
    
            print("==================================================================")
            print("가운데 손가락 roi 추출 시작좌표 : ",middle_finger_start_point)
            print("가운데 손가락 roi 추출 끝 좌표 : ",middle_finger_end_point)
            """
            roi_points = [[carpus_start_point,carpus_end_point],[four_start_point,four_end_point],[middle_finger_start_point,middle_finger_end_point]]
            #3개의 big roi를 추출했는데 좀더 잘 추출하고 싶으면 x,y값들에 오차범위를 늘려준다.
            tmp_ = []
            
            iou_points = [[],[],[]]
            #내가 추출한 roi 좌표 저장
            get_iou_points(r_img_)
            
            iou_points[0] = tmp_[0:2]
            iou_points[1] = tmp_[2:4]
            iou_points[2] = tmp_[4:6]
            print(iou_points)
            
            #roi 별로 iou 구해서 출력
            iou_cor,iou_four,iou_mid = get_iou(roi_points,iou_points)
            
            print("손목 iou : ",iou_cor*100)
            print("4개 관절 iou : ",iou_four*100)
            print("가운데 손가락 iou : ",iou_mid*100)
            
            
            if year not in dic_iou_points.keys() :
                dic_iou_points[year] = [[iou_cor*100,iou_four*100,iou_mid*100]]
            else:
                dic_iou_points[year].append([iou_cor*100,iou_four*100,iou_mid*100])
            """
            

./data/BoneAge/BA_F_10yr/
회전각도 :  1.9429170685886845
손목 관절 roi 추출 시작좌표 :  (345, 918)
손목 관절 roi 추출 끝 좌표 :  (757, 1396)
가운데 4개 관절 roi 추출 시작좌표 :  (293, 663)
가운데 4개 관절 roi 추출 끝 좌표 :  (757, 918)
가운데 손가락 roi 추출 시작좌표 :  (505, 197)
가운데 손가락 roi 추출 끝 좌표 :  (646, 684)
가운데 손가락 roi 추출 시작좌표 :  (505, 197)
가운데 손가락 roi 추출 끝 좌표 :  (646, 684)
회전각도 :  2.562576574259424
손목 관절 roi 추출 시작좌표 :  (394, 890)
손목 관절 roi 추출 끝 좌표 :  (890, 1303)
가운데 4개 관절 roi 추출 시작좌표 :  (334, 622)
가운데 4개 관절 roi 추출 끝 좌표 :  (890, 890)
가운데 손가락 roi 추출 시작좌표 :  (595, 133)
가운데 손가락 roi 추출 끝 좌표 :  (749, 623)
가운데 손가락 roi 추출 시작좌표 :  (595, 133)
가운데 손가락 roi 추출 끝 좌표 :  (749, 623)
회전각도 :  0.7053843540393103
손목 관절 roi 추출 시작좌표 :  (368, 841)
손목 관절 roi 추출 끝 좌표 :  (789, 1234)
가운데 4개 관절 roi 추출 시작좌표 :  (312, 560)
가운데 4개 관절 roi 추출 끝 좌표 :  (789, 841)
가운데 손가락 roi 추출 시작좌표 :  (504, 70)
가운데 손가락 roi 추출 끝 좌표 :  (660, 566)
가운데 손가락 roi 추출 시작좌표 :  (504, 70)
가운데 손가락 roi 추출 끝 좌표 :  (660, 566)
회전각도 :  7.086478752888453
손목 관절 roi 추출 시작좌표 :  (376, 942)
손목 관절 roi 추출 끝 좌표 :

회전각도 :  1.2763658246356755
손목 관절 roi 추출 시작좌표 :  (431, 907)
손목 관절 roi 추출 끝 좌표 :  (857, 1207)
가운데 4개 관절 roi 추출 시작좌표 :  (366, 545)
가운데 4개 관절 roi 추출 끝 좌표 :  (857, 907)
가운데 손가락 roi 추출 시작좌표 :  (538, 101)
가운데 손가락 roi 추출 끝 좌표 :  (691, 555)
가운데 손가락 roi 추출 시작좌표 :  (538, 101)
가운데 손가락 roi 추출 끝 좌표 :  (691, 555)
회전각도 :  2.340714191978009
손목 관절 roi 추출 시작좌표 :  (447, 943)
손목 관절 roi 추출 끝 좌표 :  (922, 1309)
가운데 4개 관절 roi 추출 시작좌표 :  (379, 666)
가운데 4개 관절 roi 추출 끝 좌표 :  (922, 943)
가운데 손가락 roi 추출 시작좌표 :  (629, 214)
가운데 손가락 roi 추출 끝 좌표 :  (786, 680)
가운데 손가락 roi 추출 시작좌표 :  (629, 214)
가운데 손가락 roi 추출 끝 좌표 :  (786, 680)
회전각도 :  1.215182783018534
손목 관절 roi 추출 시작좌표 :  (471, 1043)
손목 관절 roi 추출 끝 좌표 :  (925, 1506)
가운데 4개 관절 roi 추출 시작좌표 :  (400, 789)
가운데 4개 관절 roi 추출 끝 좌표 :  (925, 1043)
가운데 손가락 roi 추출 시작좌표 :  (658, 334)
가운데 손가락 roi 추출 끝 좌표 :  (795, 795)
가운데 손가락 roi 추출 시작좌표 :  (658, 334)
가운데 손가락 roi 추출 끝 좌표 :  (795, 795)
회전각도 :  5.0584521242637095
손목 관절 roi 추출 시작좌표 :  (458, 1020)
손목 관절 roi 추출 끝 좌표 :  (938, 1755)
가운데 4개 

회전각도 :  0.6939929979515256
손목 관절 roi 추출 시작좌표 :  (485, 975)
손목 관절 roi 추출 끝 좌표 :  (915, 1424)
가운데 4개 관절 roi 추출 시작좌표 :  (412, 686)
가운데 4개 관절 roi 추출 끝 좌표 :  (915, 975)
가운데 손가락 roi 추출 시작좌표 :  (625, 186)
가운데 손가락 roi 추출 끝 좌표 :  (765, 700)
가운데 손가락 roi 추출 시작좌표 :  (625, 186)
가운데 손가락 roi 추출 끝 좌표 :  (765, 700)
회전각도 :  0.484178160480937
손목 관절 roi 추출 시작좌표 :  (305, 849)
손목 관절 roi 추출 끝 좌표 :  (761, 1265)
가운데 4개 관절 roi 추출 시작좌표 :  (259, 558)
가운데 4개 관절 roi 추출 끝 좌표 :  (761, 849)
가운데 손가락 roi 추출 시작좌표 :  (496, 99)
가운데 손가락 roi 추출 끝 좌표 :  (627, 577)
가운데 손가락 roi 추출 시작좌표 :  (496, 99)
가운데 손가락 roi 추출 끝 좌표 :  (627, 577)
회전각도 :  1.5481576989779597
손목 관절 roi 추출 시작좌표 :  (444, 908)
손목 관절 roi 추출 끝 좌표 :  (864, 1381)
가운데 4개 관절 roi 추출 시작좌표 :  (377, 623)
가운데 4개 관절 roi 추출 끝 좌표 :  (864, 908)
가운데 손가락 roi 추출 시작좌표 :  (580, 162)
가운데 손가락 roi 추출 끝 좌표 :  (707, 634)
가운데 손가락 roi 추출 시작좌표 :  (580, 162)
가운데 손가락 roi 추출 끝 좌표 :  (707, 634)
회전각도 :  1.6998919335324842
손목 관절 roi 추출 시작좌표 :  (480, 956)
손목 관절 roi 추출 끝 좌표 :  (914, 1405)
가운데 4개 관절 r

회전각도 :  1.5955701288874025
손목 관절 roi 추출 시작좌표 :  (411, 1033)
손목 관절 roi 추출 끝 좌표 :  (876, 1467)
가운데 4개 관절 roi 추출 시작좌표 :  (349, 729)
가운데 4개 관절 roi 추출 끝 좌표 :  (876, 1033)
가운데 손가락 roi 추출 시작좌표 :  (603, 266)
가운데 손가락 roi 추출 끝 좌표 :  (754, 734)
가운데 손가락 roi 추출 시작좌표 :  (603, 266)
가운데 손가락 roi 추출 끝 좌표 :  (754, 734)
회전각도 :  1.6706532713999707
손목 관절 roi 추출 시작좌표 :  (433, 913)
손목 관절 roi 추출 끝 좌표 :  (864, 1341)
가운데 4개 관절 roi 추출 시작좌표 :  (368, 636)
가운데 4개 관절 roi 추출 끝 좌표 :  (864, 913)
가운데 손가락 roi 추출 시작좌표 :  (551, 150)
가운데 손가락 roi 추출 끝 좌표 :  (704, 668)
가운데 손가락 roi 추출 시작좌표 :  (551, 150)
가운데 손가락 roi 추출 끝 좌표 :  (704, 668)
회전각도 :  4.2452622844677705
손목 관절 roi 추출 시작좌표 :  (438, 1021)
손목 관절 roi 추출 끝 좌표 :  (863, 1714)
가운데 4개 관절 roi 추출 시작좌표 :  (372, 723)
가운데 4개 관절 roi 추출 끝 좌표 :  (863, 1021)
가운데 손가락 roi 추출 시작좌표 :  (592, 242)
가운데 손가락 roi 추출 끝 좌표 :  (745, 743)
가운데 손가락 roi 추출 시작좌표 :  (592, 242)
가운데 손가락 roi 추출 끝 좌표 :  (745, 743)
회전각도 :  1.0867814529981956
손목 관절 roi 추출 시작좌표 :  (402, 979)
손목 관절 roi 추출 끝 좌표 :  (920, 1461)
가운데 

회전각도 :  2.939633619176348
손목 관절 roi 추출 시작좌표 :  (411, 916)
손목 관절 roi 추출 끝 좌표 :  (861, 1360)
가운데 4개 관절 roi 추출 시작좌표 :  (349, 639)
가운데 4개 관절 roi 추출 끝 좌표 :  (861, 916)
가운데 손가락 roi 추출 시작좌표 :  (554, 133)
가운데 손가락 roi 추출 끝 좌표 :  (724, 647)
가운데 손가락 roi 추출 시작좌표 :  (554, 133)
가운데 손가락 roi 추출 끝 좌표 :  (724, 647)
회전각도 :  4.708628957258867
손목 관절 roi 추출 시작좌표 :  (400, 1038)
손목 관절 roi 추출 끝 좌표 :  (887, 1835)
가운데 4개 관절 roi 추출 시작좌표 :  (340, 677)
가운데 4개 관절 roi 추출 끝 좌표 :  (887, 1038)
가운데 손가락 roi 추출 시작좌표 :  (601, 109)
가운데 손가락 roi 추출 끝 좌표 :  (771, 681)
가운데 손가락 roi 추출 시작좌표 :  (601, 109)
가운데 손가락 roi 추출 끝 좌표 :  (771, 681)
회전각도 :  7.750513303569761
손목 관절 roi 추출 시작좌표 :  (300, 913)
손목 관절 roi 추출 끝 좌표 :  (805, 1516)
가운데 4개 관절 roi 추출 시작좌표 :  (255, 606)
가운데 4개 관절 roi 추출 끝 좌표 :  (805, 913)
가운데 손가락 roi 추출 시작좌표 :  (513, 146)
가운데 손가락 roi 추출 끝 좌표 :  (662, 619)
가운데 손가락 roi 추출 시작좌표 :  (513, 146)
가운데 손가락 roi 추출 끝 좌표 :  (662, 619)
회전각도 :  0.8836694577929762
손목 관절 roi 추출 시작좌표 :  (497, 934)
손목 관절 roi 추출 끝 좌표 :  (932, 1328)
가운데 4개 관절

...

회전각도 :  1.1899489321424426
손목 관절 roi 추출 시작좌표 :  (336, 893)
손목 관절 roi 추출 끝 좌표 :  (735, 1249)
가운데 4개 관절 roi 추출 시작좌표 :  (285, 612)
가운데 4개 관절 roi 추출 끝 좌표 :  (735, 893)
가운데 손가락 roi 추출 시작좌표 :  (455, 177)
가운데 손가락 roi 추출 끝 좌표 :  (605, 619)
가운데 손가락 roi 추출 시작좌표 :  (455, 177)
가운데 손가락 roi 추출 끝 좌표 :  (605, 619)
회전각도 :  7.881865539368093
손목 관절 roi 추출 시작좌표 :  (402, 938)
손목 관절 roi 추출 끝 좌표 :  (816, 1601)
가운데 4개 관절 roi 추출 시작좌표 :  (341, 630)
가운데 4개 관절 roi 추출 끝 좌표 :  (816, 938)
가운데 손가락 roi 추출 시작좌표 :  (564, 182)
가운데 손가락 roi 추출 끝 좌표 :  (708, 632)
가운데 손가락 roi 추출 시작좌표 :  (564, 182)
가운데 손가락 roi 추출 끝 좌표 :  (708, 632)
회전각도 :  1.615813522669896
손목 관절 roi 추출 시작좌표 :  (338, 870)
손목 관절 roi 추출 끝 좌표 :  (834, 1232)
가운데 4개 관절 roi 추출 시작좌표 :  (287, 557)
가운데 4개 관절 roi 추출 끝 좌표 :  (834, 870)
가운데 손가락 roi 추출 시작좌표 :  (559, 120)
가운데 손가락 roi 추출 끝 좌표 :  (699, 568)
가운데 손가락 roi 추출 시작좌표 :  (559, 120)
가운데 손가락 roi 추출 끝 좌표 :  (699, 568)
회전각도 :  0.08475700846109646
손목 관절 roi 추출 시작좌표 :  (497, 987)
손목 관절 roi 추출 끝 좌표 :  (898, 1346)
가운데 4개 관절

회전각도 :  0.8243428208451462
손목 관절 roi 추출 시작좌표 :  (449, 1018)
손목 관절 roi 추출 끝 좌표 :  (868, 1353)
가운데 4개 관절 roi 추출 시작좌표 :  (381, 691)
가운데 4개 관절 roi 추출 끝 좌표 :  (868, 1018)
가운데 손가락 roi 추출 시작좌표 :  (590, 275)
가운데 손가락 roi 추출 끝 좌표 :  (735, 692)
가운데 손가락 roi 추출 시작좌표 :  (590, 275)
가운데 손가락 roi 추출 끝 좌표 :  (735, 692)
회전각도 :  5.338496442969131
손목 관절 roi 추출 시작좌표 :  (448, 863)
손목 관절 roi 추출 끝 좌표 :  (879, 1366)
가운데 4개 관절 roi 추출 시작좌표 :  (380, 548)
가운데 4개 관절 roi 추출 끝 좌표 :  (879, 863)
가운데 손가락 roi 추출 시작좌표 :  (581, 85)
가운데 손가락 roi 추출 끝 좌표 :  (739, 566)
가운데 손가락 roi 추출 시작좌표 :  (581, 85)
가운데 손가락 roi 추출 끝 좌표 :  (739, 566)
회전각도 :  2.914631327487257
손목 관절 roi 추출 시작좌표 :  (387, 917)
손목 관절 roi 추출 끝 좌표 :  (833, 1300)
가운데 4개 관절 roi 추출 시작좌표 :  (328, 627)
가운데 4개 관절 roi 추출 끝 좌표 :  (833, 917)
가운데 손가락 roi 추출 시작좌표 :  (546, 108)
가운데 손가락 roi 추출 끝 좌표 :  (705, 658)
가운데 손가락 roi 추출 시작좌표 :  (546, 108)
가운데 손가락 roi 추출 끝 좌표 :  (705, 658)
회전각도 :  3.635691443105401
손목 관절 roi 추출 시작좌표 :  (339, 882)
손목 관절 roi 추출 끝 좌표 :  (1068, 1487)
가운데 4개 관절 

손목 관절 roi 추출 시작좌표 :  (426, 1072)
손목 관절 roi 추출 끝 좌표 :  (888, 1755)
가운데 4개 관절 roi 추출 시작좌표 :  (362, 734)
가운데 4개 관절 roi 추출 끝 좌표 :  (888, 1072)
가운데 손가락 roi 추출 시작좌표 :  (602, 299)
가운데 손가락 roi 추출 끝 좌표 :  (756, 740)
가운데 손가락 roi 추출 시작좌표 :  (602, 299)
가운데 손가락 roi 추출 끝 좌표 :  (756, 740)
회전각도 :  3.7046274420771255
손목 관절 roi 추출 시작좌표 :  (503, 979)
손목 관절 roi 추출 끝 좌표 :  (901, 1682)
가운데 4개 관절 roi 추출 시작좌표 :  (427, 653)
가운데 4개 관절 roi 추출 끝 좌표 :  (901, 979)
가운데 손가락 roi 추출 시작좌표 :  (644, 223)
가운데 손가락 roi 추출 끝 좌표 :  (785, 659)
가운데 손가락 roi 추출 시작좌표 :  (644, 223)
가운데 손가락 roi 추출 끝 좌표 :  (785, 659)
회전각도 :  8.38139802136493
손목 관절 roi 추출 시작좌표 :  (401, 874)
손목 관절 roi 추출 끝 좌표 :  (776, 1449)
가운데 4개 관절 roi 추출 시작좌표 :  (340, 618)
가운데 4개 관절 roi 추출 끝 좌표 :  (776, 874)
가운데 손가락 roi 추출 시작좌표 :  (518, 167)
가운데 손가락 roi 추출 끝 좌표 :  (668, 620)
가운데 손가락 roi 추출 시작좌표 :  (518, 167)
가운데 손가락 roi 추출 끝 좌표 :  (668, 620)
회전각도 :  3.3524072471030593
손목 관절 roi 추출 시작좌표 :  (420, 932)
손목 관절 roi 추출 끝 좌표 :  (814, 1458)
가운데 4개 관절 roi 추출 시작좌표 :  (357, 630)

회전각도 :  0.15871366471846216
손목 관절 roi 추출 시작좌표 :  (449, 1008)
손목 관절 roi 추출 끝 좌표 :  (879, 1352)
가운데 4개 관절 roi 추출 시작좌표 :  (381, 662)
가운데 4개 관절 roi 추출 끝 좌표 :  (879, 1008)
가운데 손가락 roi 추출 시작좌표 :  (582, 238)
가운데 손가락 roi 추출 끝 좌표 :  (738, 672)
가운데 손가락 roi 추출 시작좌표 :  (582, 238)
가운데 손가락 roi 추출 끝 좌표 :  (738, 672)
회전각도 :  0.8196261968644336
손목 관절 roi 추출 시작좌표 :  (319, 843)
손목 관절 roi 추출 끝 좌표 :  (850, 1305)
가운데 4개 관절 roi 추출 시작좌표 :  (271, 599)
가운데 4개 관절 roi 추출 끝 좌표 :  (850, 843)
가운데 손가락 roi 추출 시작좌표 :  (543, 104)
가운데 손가락 roi 추출 끝 좌표 :  (695, 600)
가운데 손가락 roi 추출 시작좌표 :  (543, 104)
가운데 손가락 roi 추출 끝 좌표 :  (695, 600)
회전각도 :  1.0715946525193374
손목 관절 roi 추출 시작좌표 :  (497, 924)
손목 관절 roi 추출 끝 좌표 :  (960, 1317)
가운데 4개 관절 roi 추출 시작좌표 :  (422, 642)
가운데 4개 관절 roi 추출 끝 좌표 :  (960, 924)
가운데 손가락 roi 추출 시작좌표 :  (653, 185)
가운데 손가락 roi 추출 끝 좌표 :  (825, 664)
가운데 손가락 roi 추출 시작좌표 :  (653, 185)
가운데 손가락 roi 추출 끝 좌표 :  (825, 664)
회전각도 :  2.0149879118324066
손목 관절 roi 추출 시작좌표 :  (377, 1047)
손목 관절 roi 추출 끝 좌표 :  (861, 1536)
가운데 

회전각도 :  0.3472428970857777
손목 관절 roi 추출 시작좌표 :  (294, 909)
손목 관절 roi 추출 끝 좌표 :  (786, 1300)
가운데 4개 관절 roi 추출 시작좌표 :  (249, 656)
가운데 4개 관절 roi 추출 끝 좌표 :  (786, 909)
가운데 손가락 roi 추출 시작좌표 :  (516, 206)
가운데 손가락 roi 추출 끝 좌표 :  (661, 672)
가운데 손가락 roi 추출 시작좌표 :  (516, 206)
가운데 손가락 roi 추출 끝 좌표 :  (661, 672)
회전각도 :  6.305008588211692
손목 관절 roi 추출 시작좌표 :  (389, 1025)
손목 관절 roi 추출 끝 좌표 :  (825, 1676)
가운데 4개 관절 roi 추출 시작좌표 :  (330, 728)
가운데 4개 관절 roi 추출 끝 좌표 :  (825, 1025)
가운데 손가락 roi 추출 시작좌표 :  (562, 235)
가운데 손가락 roi 추출 끝 좌표 :  (708, 732)
가운데 손가락 roi 추출 시작좌표 :  (562, 235)
가운데 손가락 roi 추출 끝 좌표 :  (708, 732)
회전각도 :  1.7208148336728613
손목 관절 roi 추출 시작좌표 :  (446, 998)
손목 관절 roi 추출 끝 좌표 :  (886, 1340)
가운데 4개 관절 roi 추출 시작좌표 :  (379, 698)
가운데 4개 관절 roi 추출 끝 좌표 :  (886, 998)
가운데 손가락 roi 추출 시작좌표 :  (617, 252)
가운데 손가락 roi 추출 끝 좌표 :  (769, 706)
가운데 손가락 roi 추출 시작좌표 :  (617, 252)
가운데 손가락 roi 추출 끝 좌표 :  (769, 706)
회전각도 :  1.0301035670068899
손목 관절 roi 추출 시작좌표 :  (454, 974)
손목 관절 roi 추출 끝 좌표 :  (926, 1352)
가운데 4개 

In [209]:
copy_dic_iou = dic_iou_points.copy()

In [218]:
#roi별 iou 출력
c_error = 0
f_error = 0
m_error = 0

for key,value in copy_dic_iou.items():
    
    for v in value :
        if v[0] < 50:
            c_error+=1
        if v[1] < 50:
            f_error+=1
        if v[2] < 50:
            m_error+=1
       

print(c_error,f_error,m_error)
print("정상적인 iou 비율 : ",(1 - (c_error/58))*100,(1 - (f_error/58))*100,(1 - (m_error/58))*100 )

2 0 0
정상적인 iou 비율 :  96.55172413793103 100.0 100.0


In [105]:
def get_x_data(img):
    r_img_ = np.copy(img)
    img_ = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    height, width = img_.shape
    img_ = img_[0:(int)(height*0.9),0:(int)(width*0.95)]
    ret, img = cv2.threshold(img_, img.mean()*0.99, 255, cv2.THRESH_BINARY)

    #좌표 저장공간 설정(첫번째 공간에는 무게중심점, 두번째 공간에는 start점, 세번째에는 far점) 
    dots = [[],[],[]]

    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    max_cnt = max(contours, key=cv2.contourArea)

    mask = np.zeros(img.shape, dtype=np.uint8)

    #컨투어 구하고 구한 컨투어로 이미지 그리기
    cv2.drawContours(mask, [max_cnt], -1, (255,255,255), -1)

    #이미지 팽창
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    mask = cv2.dilate(mask,k)


    #회전하기 위해서 가운데 손가락이 가장 길기때문에 가장 먼저 흰색을 나타내는 좌표를 찾아 낸다.
    first_255_x_point = 0
    first_255_y_point = 0

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_cnt = max(contours, key=cv2.contourArea)


    #무게중심점 구하기 일단 
    M = cv2.moments(max_cnt)
    center = ( int(M['m10'] / M['m00']), int(M['m01'] / M['m00']) )

    for y,x_r in enumerate(mask) :
        if 255 in x_r:
            x_255_indexs = np.where(x_r == 255)[0]

            x_255_mid_index = x_255_indexs[(int)(len(x_255_indexs)/2)]
            first_255_x_point = x_255_mid_index

            first_255_y_point = y
            break


    #width 의 중간값의 x값과 가장 먼저 흰색을 나타내는 좌표의 x값을 비교한다.
    #그리고 center 값을 기준으로 회전한다.

    half_h, half_w = center[1], center[0]
    ry = center[1]-first_255_y_point 
    rx = abs(first_255_x_point-half_w)
    radian = math.atan2(ry,rx)
    degree = 90 - (radian * 180 / math.pi)
    print("회전각도 : ", degree)

    if degree > 3:
        if first_255_x_point < half_w :
            mask = rotate_image(mask,360-degree) 
            r_img_ = rotate_image(img_,360-degree)
        else:
            mask = rotate_image(mask,degree) 
            r_img_ = rotate_image(img_,degree) 


    #이미지 외부 컨투어 구하기
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_cnt = max(contours, key=cv2.contourArea)


    #무게중심점 구하기 일단 
    M = cv2.moments(max_cnt)
    center = ( int(M['m10'] / M['m00']), int(M['m01'] / M['m00']) )


    #cv2.circle(img_,center,0,[255,255,255],10)
    dots[0].append(center)
    # 볼록한 지점 구하기
    hull = cv2.convexHull(max_cnt, returnPoints = False)
    hull1 = cv2.convexHull(max_cnt)
    # 오목한 지점 구하기
    defects = cv2.convexityDefects(max_cnt, hull)

    #거리를 저장 할 수 있는 공간 생성
    di =[]

    for index in range(defects.shape[0]):
        #시작점,끝점,far점,거리 할당
        start_point,end_point,far_point,distance = defects[index,0]

        far = tuple(max_cnt[far_point][0])
        start = tuple(max_cnt[start_point][0])
        end = tuple(max_cnt[end_point][0])

        #cv2.circle(r_img_,far,2,[255,255,255],10)
        #cv2.circle(r_img_,start,2,[255,255,255],10)

        #cv2.circle(r_img_,end,2,[255,255,255],10)
        #거리를 저장
        di.append(distance)
        dots[1].append(start)
        dots[2].append(far)

    #더 쉽게 불러오기 위해서 far, start로 분리
    far_pt = np.array(dots[2])
    start_pt = np.array(dots[1])


    far_xrange =[]
    far_yrange = []
    far_miny = 1000
    far_maxy = 0
    start_miny = 1000
    #가장 오목하게 들어가 있는 부분을 찾기 위해서 sort(내림차순)
    di =np.array(di)
    s_di=np.sort(di)[::-1]
    #내림차순된 거리들을 6개만 뽑아내기 위해서 slice
    for i in list(s_di[:6]):
        index = np.where(di ==i)[0]
        #6개의 좌표들 중에서 가장 최저의 y 값을 찾는다. (손목쪽 roi에 필요)
        far_miny = min(far_miny,far_pt[index[0]][1])
        #6개의 좌표들 중에서 가장 최고의 y 값을 찾는다.
        far_maxy = max(far_maxy,far_pt[index[0]][1])

        #가장 오목한 지점 6개의 좌표를 출력
        #cv2.circle(img_,np.array(far_pt[index[0]]),2,[255,255,255],5)

        #좌표들이 x,y로 나눠져 있어서 쉽게 비교하기 위해서 x,y끼리 나눈다.
        far_xrange.append(far_pt[index[0]][0])
        far_yrange.append(far_pt[index[0]][1])

    #far_xrange를 오름차순으로 정렬
    sorted_far_xrange = np.sort(far_xrange)
    sorted_far_yrange = np.sort(far_yrange)

    carpus_start_point = ((int)(np.sort(far_xrange)[0]),(int)(center[1]*1.05))
    carpus_end_point = (np.sort(far_xrange)[-1],(int)(far_maxy*1.09))
    #손목뼈 부분 roi 를 하기 위해서는 가장 오목하게 들어가 있는 부분중에서 가장 최저 x값(xrange[0])과 center값의 y값을 시작점으로
    #끝점으로는 가장 오목하게 들어가 있는 부분들 중에서 가장 최고 x값(xrange[-1])과 오목하게 들어간 점중에서 가장 최고 y값을 준다.
    #cv2.rectangle(r_img_,carpus_start_point,carpus_end_point,[255,255,255],5)

    print("==================================================================")
    print("손목 관절 roi 추출 시작좌표 : ",carpus_start_point)
    print("손목 관절 roi 추출 끝 좌표 : ",carpus_end_point)

    #손목뼈 위쪽에 있는 관절 4개를 추출하기 위해서는 오목하게 들어가 있는 부분중에서 가장 최저 x값(xrange[0])과 far_miny 값을 y값으로
    #끝점으로는 가장 오목하게 들어가 있는 부분들 중에서 가장 최고 x값(xrange[-1])과 center의 y값을 준다.

    four_start_point = ((int)(np.sort(far_xrange)[0]*0.85),far_miny)
    four_end_point = (np.sort(far_xrange)[-1],(int)(center[1]*1.05))
    #cv2.rectangle(r_img_,four_start_point,four_end_point,[255,255,255],5)

    print("==================================================================")
    print("가운데 4개 관절 roi 추출 시작좌표 : ",four_start_point)
    print("가운데 4개 관절 roi 추출 끝 좌표 : ",four_end_point)
    #가운데 손가락을 추출하기 위해서 start_pt의 x 좌표가 sorted_far_xrange 에서 3번째와 4번째 값 사이에 있어야 한다.
    middle_index = np.where((start_pt[:,0]<=(int)(sorted_far_xrange[3])) & (start_pt[:,0]>=sorted_far_xrange[2]))[0]
    #위에 조건식으로 나온 인덱스를 start_point에 대입하면 만족하는 좌표들이 여러개 나올것이다.
    middle_points = start_pt[middle_index]


    for point in middle_points:
        #가운데 손가락 사이에 있는 좌표들 중에서 최저 y 값을 찾는다.
        start_miny = min(start_miny,point[1])



    start_maxy = max(far_yrange[np.where(far_xrange == sorted_far_xrange[3])[0][0]],far_yrange[np.where(far_xrange == sorted_far_xrange[2])[0][0]])
    #시작 좌표로는 x값으로 sorted_far_xrange 에서 3번째와 y 값으로는 최저y 값을 주고 
    #마지막 좌표로는 x값으로 sorted_far_xrange 에서 4번째와 y값으로는 3번째 와 4번째 좌표의 최고 y 값을 준다.
    middle_finger_start_point = (sorted_far_xrange[2],start_miny)
    middle_finger_end_point = (sorted_far_xrange[3],start_maxy)


    print("==================================================================")
    print("가운데 손가락 roi 추출 시작좌표 : ",middle_finger_start_point)
    print("가운데 손가락 roi 추출 끝 좌표 : ",middle_finger_end_point)
    #cv2.rectangle(r_img_,middle_finger_start_point,middle_finger_end_point,[255,255,255],5)


       #새끼손가락 시작점 x좌표
    little_finger_x = ()
    little_finger_y = ()

    min_little_finger_x = hull1[0][0][0]
    for i in range(1, len(hull1)):
        if min_little_finger_x > hull1[i][0][0]:
            min_little_finger_x = hull1[i][0][0]
            little_finger_x, little_finger_y = hull1[i][0]

    #엄지손가락 시작점 좌표
    thumb_x = ()
    thumb_y = ()

    max_thumb_x = hull1[0][0][0]
    for i in range(1,len(hull1)):
        if max_thumb_x < hull1[i][0][0]:
            max_thumb_x = hull1[i][0][0]            
            thumb_x, thumb_y = hull1[i][0] 


    # 새끼손가락 endpoint x 좌표
    little_finger_endpoint_x =[]
    for x, y in zip(far_xrange, far_yrange):
        if y < int(center[1]*0.9) :
            little_finger_endpoint_x.append(x)

    # 엄지손가락 endpoint
    thumb_endpoint_list =[]
    for x, y in zip(far_xrange, far_yrange):
        if y > int(center[1]):
            thumb_endpoint_list.append([x,y])

    thumb_endpoint =[]
    max_x = thumb_endpoint_list[0][0]
    if len(thumb_endpoint_list) == 1:
        thumb_endpoint = thumb_endpoint_list
    for i in range(1, len(thumb_endpoint_list)):       
        if max_x < thumb_endpoint_list[i][0]:
            max_x = thumb_endpoint_list[i][0]
            thumb_endpoint.append(thumb_endpoint_list[i])
        else:
            thumb_endpoint.append(thumb_endpoint_list[0])

     #새끼손가락 ROI
    little_finger_start_point = (int(little_finger_x*0.5), int(little_finger_y * 0.8))
    little_finger_end_point = (min(little_finger_endpoint_x),(int)(center[1]*1.05))
    cv2.rectangle(r_img_,little_finger_start_point, little_finger_end_point,[255,255,255],2)
    print("==================================================================")
    print("새끼손가락 roi 추출 시작좌표 : ",little_finger_start_point)
    print("새끼손가락 roi 추출 끝 좌표 : ",little_finger_end_point)


    #엄지손가락 ROI
    thumb_start_point = (int(thumb_endpoint[0][0]*1.1), int(thumb_y*0.9))
    thumb_end_point = (int(thumb_x * 1.02), int(thumb_endpoint[0][1]))
    cv2.rectangle(r_img_,thumb_start_point, thumb_end_point,[255,255,255],2)
    print("==================================================================")
    print("엄지손가락 roi 추출 시작좌표 : ",thumb_start_point)
    print("엄지손가락 roi 추출 끝 좌표 : ",thumb_end_point)


    
    optimzed_imgs = crop_img(r_img_,(carpus_start_point,carpus_end_point),(four_start_point, four_end_point),(middle_finger_start_point,middle_finger_end_point),(thumb_start_point,thumb_end_point),(little_finger_start_point,little_finger_end_point))
    if len(optimzed_imgs) > 0:
        optimzed_imgs = np.array(optimzed_imgs)

        X_data.append(optimzed_imgs.reshape(256,256,-1))
        y_data.append(num_year)  
    
    return np.array(X_data),np.array(y_data)

In [130]:
pz = cv2.imread('./data/BoneAge/BA_M_15yr/BA_M_15yr/625_M.jpg')
x,y = get_x_data(pz)

회전각도 :  10.905621141604172
손목 관절 roi 추출 시작좌표 :  (462, 1121)
손목 관절 roi 추출 끝 좌표 :  (1024, 1528)
가운데 4개 관절 roi 추출 시작좌표 :  (392, 709)
가운데 4개 관절 roi 추출 끝 좌표 :  (1024, 1121)
가운데 손가락 roi 추출 시작좌표 :  (677, 166)
가운데 손가락 roi 추출 끝 좌표 :  (860, 726)
새끼손가락 roi 추출 시작좌표 :  (0, 1404)
새끼손가락 roi 추출 끝 좌표 :  (528, 1121)
엄지손가락 roi 추출 시작좌표 :  (985, 992)
엄지손가락 roi 추출 끝 좌표 :  (1518, 1402)
None


In [131]:
p_y = model.predict(x.reshape(-1,256,256,3))